In [1]:
def euclidean_dist(a, b):
    return np.sqrt(np.sum((a - b)**2))

### Silhouette Score (manual implementation)

This function computes the **average Silhouette Score** without sklearn.

For each point:

- **a(i)** → mean distance to points in the **same** cluster  
- **b(i)** → mean distance to points in the **nearest other** cluster  

Silhouette value:

$$
s(i)=\frac{b(i)-a(i)}{\max(a(i),\, b(i))}
$$

Then it returns the **average of all s(i)**.

Edge cases handled:
- single-point clusters → score = 0  
- only one cluster → score = 0


In [ ]:
def silhouette_score_manual(X, labels):
    """
    Computes the mean Silhouette Coefficient of all samples.
    """
    n_samples = X.shape[0]
    unique_labels = np.unique(labels) #the labels are the cluster assignments produced by your clustering algorithm.
    s_values = []

    for i in range(n_samples):
        point = X[i]
        label = labels[i] # cluster assignment of the point
        
        # Calculate a(i): Mean distance to other points in the SAME cluster
        same_cluster_mask = (labels == label)
        # Exclude the point itself
        other_points_indices = np.where(same_cluster_mask)[0] 
        other_points_indices = other_points_indices[other_points_indices != i] # Exclude self
        
        if len(other_points_indices) == 0: # Only one point in cluster
            s_values.append(0) # Silhouette is undefined, assign 0
            continue
            
        a_i = np.mean([euclidean_dist(point, X[idx]) for idx in other_points_indices])
        
        # Calculate b(i): Mean distance to points in the NEAREST neighboring cluster
        b_i = np.inf
        for other_label in unique_labels:
            if other_label == label:
                continue
            
            other_cluster_points = X[labels == other_label]
            if len(other_cluster_points) == 0: # No points in this cluster
                continue
                
            mean_dist = np.mean([euclidean_dist(point, p) for p in other_cluster_points])
            b_i = min(b_i, mean_dist)
        
        # Silhouette for this point
        if b_i == np.inf: # Only one cluster exists
            s = 0
        else:
            s = (b_i - a_i) / max(a_i, b_i)
        s_values.append(s)

    return np.mean(s_values)

### Davies–Bouldin Index (manual implementation)

This function computes the **Davies–Bouldin Index (DBI)** — a clustering metric where  
**lower values indicate better clustering**.

For each cluster:

1️-Compute the **centroid**  
2️-Compute its **scatter** = average distance of points to the centroid  

Then, for every pair of clusters \(i,j\):

- compute centroid distance \(M_{ij}\)
- compute similarity ratio:

$$
R_{ij} = \frac{S_i + S_j}{M_{ij}}
$$

For each cluster, take the **maximum \(R_{ij}\)** (worst overlap), then average:

$$
DBI = \frac{1}{k}\sum_{i=1}^{k}\max_{j\ne i} R_{ij}
$$

**Lower DBI = tighter, better-separated clusters.**


In [ ]:
def davies_bouldin_manual(X, labels):
    unique_labels = np.unique(labels)
    n_clusters = len(unique_labels)
    
    #  Calculate Centroids and Scatter (Avg dist to centroid)
    centroids = []
    scatters = []
    
    for k in unique_labels:
        cluster_points = X[labels == k]
        centroid = np.mean(cluster_points, axis=0) #axis=0 for mean of each feature
        centroids.append(centroid)
        
        # Average distance from points to centroid
        avg_dist = np.mean([euclidean_dist(p, centroid) for p in cluster_points])
        scatters.append(avg_dist)
        
    # Compute DB Index
    db_score = 0
    for i in range(n_clusters):
        max_ratio = -np.inf
        for j in range(n_clusters):
            if i == j: # skip same cluster
                continue
            
            # Distance between centroids
            dist_centroids = euclidean_dist(centroids[i], centroids[j])
            
            # Ratio: (Scatter_i + Scatter_j) / Distance_ij
            ratio = (scatters[i] + scatters[j]) / dist_centroids
            
            if ratio > max_ratio:
                max_ratio = ratio
        
        db_score += max_ratio
        
    return db_score / n_clusters

### Calinski–Harabasz Index (manual implementation)

This function computes the **Calinski–Harabasz Index (CHI)**, also called the *Variance Ratio Criterion*.  
It compares **between-cluster dispersion** to **within-cluster dispersion**.

Steps:

1. Compute the **global mean** of all data points.  
2. For each cluster:
   - compute its **centroid**
   - accumulate:
     - \(W_k\): sum of squared distances of points to their centroid  
     - \(B_k\): weighted squared distance of the centroid to the global mean

The index is:

$$
CHI = \frac{B_k / (k-1)}{W_k / (n-k)}
$$

Interpretation:  
**Higher CHI indicates tighter clusters that are well separated.**


In [ ]:
def calinski_harabasz_manual(X, labels):
    n_samples = X.shape[0]
    unique_labels = np.unique(labels)
    n_clusters = len(unique_labels)
    
    if n_clusters < 2:
        return 0
        
    global_mean = np.mean(X, axis=0)
    
    # Within-Cluster Scatter (W_k) & Between-Cluster Scatter (B_k)
    W_k = 0
    B_k = 0
    
    for k in unique_labels: #calculate for each cluster
        cluster_points = X[labels == k]
        centroid = np.mean(cluster_points, axis=0)
        n_points = len(cluster_points)
        
        # W_k contribution: Sum of squared distances to centroid
        dists_to_centroid = np.sum((cluster_points - centroid)**2)
        W_k += dists_to_centroid
        
        # B_k contribution: Weighted squared distance of centroid to global mean
        B_k += n_points * np.sum((centroid - global_mean)**2)  #sum for each feature and weight by number of points
        
    # Formula: (B_k / (k - 1)) / (W_k / (n - k))
    ch_index = (B_k / (n_clusters - 1)) / (W_k / (n_samples - n_clusters))
    
    return ch_index

### WCSS (Within-Cluster Sum of Squares) and GMM Metrics

**WCSS / Inertia**  
Measures cluster compactness for k-means:

- For each point, compute squared distance to its cluster centroid  
- Sum over all points:

$$
WCSS = \sum_{i=1}^{n} \| x_i - \mu_{c_i} \|^2
$$

Lower WCSS indicates tighter clusters.

---

**GMM Metrics (AIC & BIC)**  
Used to evaluate Gaussian Mixture Models:

- **AIC** (Akaike Information Criterion):

$$
AIC = 2k - 2 \ln(L)
$$

- **BIC** (Bayesian Information Criterion):

$$
BIC = k \ln(n) - 2 \ln(L)
$$

Where:  
- \(k\) = number of parameters (means, covariances, weights)  
- \(L\) = log-likelihood  
- \(n\) = number of samples  

Lower AIC/BIC indicates a better model, balancing fit and complexity.


In [7]:
# WCSS (Inertia) wcss-> within cluster sum of squares
def compute_wcss(X, labels, centroids):
    wcss = 0
    for i, point in enumerate(X):
        centroid = centroids[labels[i]]
        wcss += np.sum((point - centroid)**2)
    return wcss

# GMM Metrics
def gmm_metrics(log_likelihood, n_samples, n_features, n_clusters, covariance_type='full'):
    """
    Calculates AIC and BIC for GMM.
    """
    # Number of parameters calculation (approximate for 'full' covariance)
    # Means (k*d) + Covariances (k * d*(d+1)/2) + Weights (k-1)
    n_params = (n_clusters * n_features) + \
               (n_clusters * n_features * (n_features + 1) / 2) + \
               (n_clusters - 1)
               
    # BIC = k*ln(n) - 2*ln(L)
    bic = (n_params * np.log(n_samples)) - (2 * log_likelihood)
    
    # AIC = 2k - 2*ln(L)
    aic = (2 * n_params) - (2 * log_likelihood)
    
    return aic, bic

### Purity Score (manual implementation)

Purity measures how **homogeneous clusters** are with respect to the true labels.

Steps:

1. For each predicted cluster:
   - Find the **true labels** of points in that cluster  
   - Identify the **most frequent true label**  
   - Count points correctly assigned to this label

2. Sum the counts over all clusters and divide by total samples:

$$
Purity = \frac{1}{N} \sum_{k} \max_j | c_k \cap t_j |
$$

Where:  
- \(c_k\) = points in predicted cluster \(k\)  
- \(t_j\) = points of true class \(j\)  
- \(N\) = total number of points

Purity ranges from **0 to 1**, with **1 being perfectly pure clusters**.


In [6]:
def purity_score_manual(y_true, y_pred):
    # Compute confusion matrix first (intersection counts)
    # Rows: True, Cols: Pred
    unique_true = np.unique(y_true)
    unique_pred = np.unique(y_pred)
    
    total_samples = len(y_true)
    correct_counts = 0
    
    for p in unique_pred:
        # Find indices where prediction is p
        pred_mask = (y_pred == p)
        
        # Get true labels for these points
        true_labels_in_cluster = y_true[pred_mask]
        
        if len(true_labels_in_cluster) == 0:
            continue
            
        # Find most frequent true label in this cluster
        values, counts = np.unique(true_labels_in_cluster, return_counts=True)
        max_count = np.max(counts)
        
        correct_counts += max_count
        
    return correct_counts / total_samples #true positive rate 

### Adjusted Rand Index (manual implementation)

The **Adjusted Rand Index (ARI)** measures similarity between **true labels** and **predicted clusters**, correcting for chance.

Steps:

1. Compute the **contingency table** between true classes and predicted clusters.  
2. Count all pairs of points:

- **nC2 for predicted clusters** → \( \text{tp\_plus\_fp} \)  
- **nC2 for true classes** → \( \text{tp\_plus\_fn} \)  
- **nC2 for intersections** → \( \text{tp} \)

3. Compute total pairs: \( \text{nC2(total samples)} \)  
4. Compute **expected index** (random chance) and **maximum index**  
5. Apply ARI formula:

$$
ARI = \frac{TP - \text{Expected Index}}{\text{Max Index} - \text{Expected Index}}
$$

Interpretation:  
- **1** → perfect agreement  
- **0** → random labeling  
- Can be negative → worse than random


In [5]:
from scipy.special import comb # Used only for nCr calculation

def adjusted_rand_index_manual(y_true, y_pred):
    # Helper: nC2 (combinations of 2)
    def nC2(n):
        if n < 2: return 0
        return n * (n - 1) / 2

    #  Contingency Table
    classes = np.unique(y_true)
    clusters = np.unique(y_pred)
    
    tp_plus_fp = 0 # Sum of nC2 for predicted clusters (a_i)
    tp_plus_fn = 0 # Sum of nC2 for true classes (b_j)
    tp = 0         # Sum of nC2 for intersection (n_ij)
    
    # Calculate row sums (a_i)
    for c in clusters:
        n_c = np.sum(y_pred == c)
        tp_plus_fp += nC2(n_c)
        
    # Calculate col sums (b_j)
    for k in classes:
        n_k = np.sum(y_true == k)
        tp_plus_fn += nC2(n_k)
        
    # Calculate intersections (n_ij)
    for c in clusters:
        for k in classes:
            n_ij = np.sum((y_pred == c) & (y_true == k))
            tp += nC2(n_ij)
            
    # Total pairs
    n_samples = len(y_true)
    total_pairs = nC2(n_samples)
    
    # Expected Index (by chance)
    expected_index = (tp_plus_fp * tp_plus_fn) / total_pairs
    
    # Max Index
    max_index = (tp_plus_fp + tp_plus_fn) / 2
    
    # ARI Formula
    if max_index == expected_index:
        return 0
    return (tp - expected_index) / (max_index - expected_index)

### Normalized Mutual Information (manual implementation)

**NMI** measures the similarity between **true labels** and **predicted clusters**, normalized to [0,1].

Steps:

1. Compute **entropy** of true labels \(H(Y)\) and predicted clusters \(H(C)\):

$$
H(X) = -\sum_i p_i \log(p_i)
$$

2. Compute **Mutual Information**:

$$
I(Y;C) = \sum_{y \in Y} \sum_{c \in C} P(y,c) \log \frac{P(y,c)}{P(y) P(c)}
$$

3. Normalize:

$$
NMI = \frac{2 \cdot I(Y;C)}{H(Y) + H(C)}
$$

Interpretation:  
- **1** → perfect agreement  
- **0** → no mutual information


In [8]:
def entropy(labels):
    """ Helper to compute Entropy H(X) """
    n = len(labels)
    if n == 0: return 0
    _, counts = np.unique(labels, return_counts=True)
    probs = counts / n
    return -np.sum(probs * np.log(probs + 1e-10)) # 1e-10 for numerical stability

def nmi_manual(y_true, y_pred):
    n = len(y_true)
    
    # Entropy of Class H(Y) and Cluster H(C)
    H_Y = entropy(y_true)
    H_C = entropy(y_pred)
    
    # Mutual Information I(Y; C)
    # I(Y; C) = Sum_y Sum_c P(y,c) * log( P(y,c) / (P(y)*P(c)) )
    MI = 0
    classes = np.unique(y_true)
    clusters = np.unique(y_pred)
    
    for k in classes:
        for c in clusters:
            # Intersection count
            n_kc = np.sum((y_true == k) & (y_pred == c))
            
            if n_kc > 0:
                p_kc = n_kc / n
                p_k = np.sum(y_true == k) / n
                p_c = np.sum(y_pred == c) / n
                
                MI += p_kc * np.log(p_kc / (p_k * p_c + 1e-10))
                
    # Normalized MI
    # NMI = 2 * I(Y;C) / (H(Y) + H(C))
    if (H_Y + H_C) == 0:
        return 0
    return 2 * MI / (H_Y + H_C)